Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Do the [Plotly Dash](https://dash.plot.ly/) Tutorial, Parts 1 & 2.
- [ ] Add your own stretch goal(s) !

In [25]:
# If you're in Colab...
import os, sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install required python packages:
    # pandas-profiling, version >= 2.0
    # plotly, version >= 4.0
    !pip install --upgrade pandas-profiling plotly
    
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Change into directory for module
    os.chdir('module1')

Requirement already up-to-date: pandas-profiling in /usr/local/lib/python3.6/dist-packages (2.3.0)
Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.1.1)
Reinitialized existing Git repository in /content/.git/
fatal: remote origin already exists.
From https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification
 * branch            master     -> FETCH_HEAD
Already up to date.


In [26]:
!pip install fiona
!pip install geopandas

In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv('../data/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [50]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
import geopandas
import json

nyc = geopandas.read_file('/content/NYC.geojson')
nyc.head()

,ntacode,shape_area,ntaname,shape_leng,boroname,borocode,countyfips,geometry
0,BK43,35799637.8103,Midwood,27996.5912736,Brooklyn,3,047,(POLYGON ((-73.94732672160586 40.6291665672094...
1,BK75,32629833.1149,Bedford,29992.9191744,Brooklyn,3,047,(POLYGON ((-73.94193078816201 40.7007252346955...
2,BX40,6307283.62202,Fordham South,15878.2729212,Bronx,2,005,(POLYGON ((-73.89138023380268 40.8617005882630...
3,BK88,54005019.2286,Borough Park,39247.227722,Brooklyn,3,047,(POLYGON ((-73.9760493559142 40.63127590564666...
4,BK96,32706946.4035,Rugby-Remsen Village,30957.8533949,Brooklyn,3,047,(POLYGON ((-73.90855790522774 40.6520959377945...


In [31]:
nyc['ntaname'].unique()

array(['Midwood', 'Bedford', 'Fordham South', 'Borough Park',
       'Rugby-Remsen Village', 'East Flushing', 'Woodhaven', 'Madison',
       'Auburndale', 'Williamsbridge-Olinville', 'Murray Hill',
       'East Elmhurst', 'Brownsville', 'East New York (Pennsylvania Ave)',
       'Kensington-Ocean Parkway', 'Parkchester', 'Erasmus',
       'Cambria Heights', 'East Flatbush-Farragut', 'Ocean Parkway South',
       'Starrett City', 'Morrisania-Melrose', 'Elmhurst', 'East Village',
       'Glen Oaks-Floral Park-New Hyde Park', 'Longwood', 'Yorkville',
       'Upper East Side-Carnegie Hill', 'Windsor Terrace',
       'Hammels-Arverne-Edgemere', 'Rikers Island', 'Hunts Point',
       'Jackson Heights', 'Bath Beach',
       'Old Town-Dongan Hills-South Beach', 'Flatbush',
       'Melrose South-Mott Haven North', 'Ocean Hill',
       'Morningside Heights', 'Soundview-Bruckner',
       'Allerton-Pelham Gardens', 'Jamaica Estates-Holliswood', 'Hollis',
       'Flatlands', 'East New York', 'Kings

In [48]:
from shapely.geometry import shape, Point
# depending on your version, use: from shapely.geometry import shape, Point

# load GeoJSON file containing sectors
with open('/content/NYC.geojson') as f:
    js = json.load(f)
    


# construct point based on lon/lat returned by geocoder
point = Point(-73.9667, 40.7947)

# check each polygon to see if it contains the point
for feature in js['features']:
    polygon = shape(feature['geometry'])
    if polygon.contains(point):
        print('Found containing polygon:', feature)
        print(feature['properties']['ntaname'])

Found containing polygon: {'type': 'Feature', 'properties': {'ntacode': 'MN12', 'shape_area': '34381053.2223', 'ntaname': 'Upper West Side', 'shape_leng': '29158.1326694', 'boroname': 'Manhattan', 'borocode': '1', 'countyfips': '061'}, 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[-73.96003018437096, 40.798038107896936], [-73.96052271669548, 40.79736846946238], [-73.9609797180794, 40.79673864404534], [-73.96144060655743, 40.79611082718398], [-73.96189985460958, 40.79547927006117], [-73.96235980150675, 40.79485206056069], [-73.96280590635736, 40.79423581323217], [-73.96371096541826, 40.793012934883265], [-73.96417598521327, 40.792362045027765], [-73.96468540739485, 40.79166402679886], [-73.96517705499018, 40.79099034109937], [-73.96562799538663, 40.79036611712905], [-73.96609500572451, 40.7897343897667], [-73.96655226678924, 40.78910715282558], [-73.96700977073407, 40.78847679074222], [-73.96744908373162, 40.7878607205905], [-73.96792696354473, 40.78722157112605], [-73.9683847

In [64]:
def neighborhood(long, lat):
  # construct point based on lon/lat returned by geocoder
  point = Point(long, lat)

# check each polygon to see if it contains the point
  for feature in js['features']:
    polygon = shape(feature['geometry'])
    if polygon.contains(point):
      n_name = feature['properties']['ntaname']
    
  return  n_name


longitude = -73.9667
latitude = 40.7947
n_name = neighborhood(longitude, latitude)
print(n_name)


Upper West Side
